# Installation and minimal working example of dsmpy

This notebook aims to install [dsmpy](https://github.com/afeborgeaud/dsmpy) on the binder and to run the minimal working example of computing waveforms.

## NOTE on installation
- It is easier to update the conda enviroment of `notebook` rather than newly activating the `dsm` in jupyter notebook.
- the version of obspy should be 1.3.0; otherwise it causes the error associated with 'split'
- We install gfortran using conda as well. 

## How to run this notebook
**Please run cell by cell.** `Run All Cells` will be stuck when compiling the source. Restart kernel if it's stuck.

In [ ]:
# download dsmpy package from github and dependencies
%cd ~/dsmpy_launch
!git clone https://github.com/afeborgeaud/dsmpy
!conda install --yes --prune --prefix /srv/conda/envs/notebook conda-build python=3.9 pytest numpy mpi4py matplotlib pandas conda-forge::obspy=1.3.0 conda-forge::geographiclib cartopy conda-forge::gfortran    

**NOTE:** If you want to make a new enviroment when running this notebook in local machine, you could change the prefix from the enviroment `notebook`.

### Compile the source

In [ ]:
%cd ~/dsmpy_launch/dsmpy
%pip install --no-build-isolation --no-deps -e . --prefix=/srv/conda/envs/notebook

In [ ]:
# conduct pytest
!pytest

**WARNING!**
Here we found the error of test with `test_seismicmodel.py`. The true values of `waveform_grads_0000` given in the test script is not identical to what we computed during this test. We may need to check the computation of `waveform_grads`.

In [ ]:
# back to the working directory
%cd ~/dsmpy_launch
!mkdir demo
%cd demo

## Minimal working example
### Running dsmpy from a python script

In [ ]:
import os
import matplotlib.pyplot as plt

from dsmpy import dsm, seismicmodel
from dsmpy.event import Event
from dsmpy.station import Station
from dsmpy.utils.cmtcatalog import read_catalog

root_dir = f'{os.path.expanduser("~")}/dsmpy_launch/demo' # absolute path of home directory

In [ ]:
# load gcmt catalog
catalog = read_catalog()
# get event from catalog
event = Event.event_from_catalog(
    catalog, '200707211534A')
# define station FCC
stations = [
    Station(
        name='FCC', network='CN',
        latitude=58.7592, longitude=-94.0884), 
    ]
# load (anisotropic) PREM model
seismic_model = seismicmodel.SeismicModel.prem()
tlen = 3276.8 # duration of synthetics (s)
nspc = 256 # number of points in frequency domain
sampling_hz = 20 # sampling frequency for sythetics
# create input parameters for pydsm
input = dsm.PyDSMInput.input_from_arrays(
    event, stations, seismic_model, tlen, nspc, sampling_hz)
# compute synthetics in frequency domain calling DSM Fortran
output = dsm.compute(input)
output.to_time_domain() # perform inverse FFT
output.filter(freq=0.04) # apply a 25 seconds low-pass filter
us = output.us # synthetics. us.shape = (3,nr,tlen)
ts = output.ts # time points [0, tlen]
# brackets can be used to access component and station
u_Z_FCC = output['Z', 'FCC_CN']
# to plot a three-component record section, use
output.plot()
plt.show()

In [ ]:
# to write synthetics to SAC files, use
output.write(root_path=root_dir, format='sac')

### Running dsmpy using a (Fortran) DSM input file.

In [ ]:
from dsmpy import dsm, rootdsm_sh
parameter_file = rootdsm_sh + 'AK135_SH.inf'
inputs = dsm.PyDSMInput.input_from_file(parameter_file, file_mode=2)
outputs = dsm.compute(inputs, mode=2)
outputs.to_time_domain()
us = outputs.us    # us.shape = (3,nr,tlen)
ts = outputs.ts    # len(ts) = tlen
stations = outputs.stations        # len(stations) = nr
components = outputs.components    # len(components) = 3

In [ ]:
outputs.plot()

In [ ]:
outputs.write(root_path=root_dir, format='sac')